Encoding

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data_cleaned.csv')

In [ ]:
le = LabelEncoder()
df['Type_encoded'] = le.fit_transform(df['Type'])  # L->0, M->1, H->2 

KeyError: "['TYPE'] not found in axis"

In [ ]:
df = df.drop('Type', axis=1) 
df 

,Unnamed: 0,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type,Type_encoded
0,0,298.1,308.6,1551,42.8,0,0,No Failure,2
1,1,298.2,308.7,1408,46.3,3,0,No Failure,1
2,2,298.1,308.5,1498,49.4,5,0,No Failure,1
3,3,298.2,308.6,1433,39.5,7,0,No Failure,1
4,4,298.2,308.7,1408,40.0,9,0,No Failure,1
...,...,...,...,...,...,...,...,...,...
9995,9995,298.8,308.4,1604,29.5,14,0,No Failure,2
9996,9996,298.9,308.4,1632,31.8,17,0,No Failure,0
9997,9997,299.0,308.6,1645,33.4,22,0,No Failure,2
9998,9998,299.0,308.7,1408,48.5,25,0,No Failure,0


In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [11]:

print(df['Failure Type'].value_counts())
df = df.drop('Failure Type', axis=1)

Failure Type
No Failure                  9652
Heat Dissipation Failure     112
Power Failure                 95
Overstrain Failure            78
Tool Wear Failure             45
Random Failures               18
Name: count, dtype: int64


In [12]:
#splitting
X = df.drop('Target', axis=1)  
y = df['Target'] 

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [17]:
#before handling imbalance
y_train.value_counts()

Target
0    7729
1     271
Name: count, dtype: int64

In [18]:
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [19]:
y_train_res.value_counts()

Target
0    7729
1    7729
Name: count, dtype: int64

In [20]:
#MODEL

In [ ]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

lr = LogisticRegression(
    max_iter=1000,
    class_weight = 'balanced',
    random_state = 42
)

lr.fit(X_train_res, y_train_res)

y_pred = lr.predict(X_test)
y_proba = lr.predict_proba(X_test)[:,1]

print("=== Logistic Regression Metrics ===")
print(classification_report(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC Score:", roc_auc)

=== Logistic Regression Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.85      0.91      1932
           1       0.15      0.79      0.26        68

    accuracy                           0.84      2000
   macro avg       0.57      0.82      0.58      2000
weighted avg       0.96      0.84      0.89      2000

Confusion Matrix:
 [[1634  298]
 [  14   54]]
ROC-AUC Score: 0.894782913165266


#we can see that it performs badly with class 1

In [25]:
X_train_res.columns = X_train_res.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
X_test.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

In [26]:
from xgboost import XGBClassifier


scale_pos_weight = (y_train_res==0).sum() / (y_train_res==1).sum()


xgb = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)


xgb.fit(X_train_res, y_train_res)


y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)[:,1]


print("=== XGBoost Metrics ===")
print(classification_report(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC Score:", roc_auc)


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [19:06:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


=== XGBoost Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1932
           1       0.56      0.81      0.66        68

    accuracy                           0.97      2000
   macro avg       0.78      0.89      0.82      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1889   43]
 [  13   55]]
ROC-AUC Score: 0.9626720253318719


In [27]:
#geting a bit better with class 1

In [28]:
from sklearn.svm import SVC



svm = SVC(
    class_weight='balanced',
    probability=True,
    random_state=42
)


svm.fit(X_train_res, y_train_res)


y_pred = svm.predict(X_test)
y_proba = svm.predict_proba(X_test)[:,1]


print("=== SVM Metrics ===")
print(classification_report(y_test, y_pred))


cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC Score:", roc_auc)


=== SVM Metrics ===
              precision    recall  f1-score   support

           0       1.00      0.79      0.88      1932
           1       0.13      0.90      0.23        68

    accuracy                           0.79      2000
   macro avg       0.56      0.84      0.55      2000
weighted avg       0.97      0.79      0.86      2000

Confusion Matrix:
 [[1527  405]
 [   7   61]]
ROC-AUC Score: 0.9100520643039824


In [29]:
#class 1 - really bad

In [30]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced'  
)

rf.fit(X_train_res, y_train_res)

y_pred = rf.predict(X_test)
y_proba = rf.predict_proba(X_test)[:,1]

print("=== Random Forest Metrics ===")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC Score:", roc_auc)

=== Random Forest Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1932
           1       0.43      0.75      0.55        68

    accuracy                           0.96      2000
   macro avg       0.71      0.86      0.76      2000
weighted avg       0.97      0.96      0.96      2000

Confusion Matrix:
 [[1864   68]
 [  17   51]]
ROC-AUC Score: 0.9619489099987821


In [34]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# -----------------------------
# Scale features
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# Initialize MLPClassifier
# -----------------------------
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),  # 2 hidden layers
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42
)

# Train model
mlp.fit(X_train_scaled, y_train_res)

# Predict
y_pred = mlp.predict(X_test_scaled)
y_proba = mlp.predict_proba(X_test_scaled)[:,1]

# Metrics
print("=== MLPClassifier Metrics ===")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC Score:", roc_auc)



=== MLPClassifier Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      1932
           1       0.33      0.75      0.46        68

    accuracy                           0.94      2000
   macro avg       0.66      0.85      0.71      2000
weighted avg       0.97      0.94      0.95      2000

Confusion Matrix:
 [[1829  103]
 [  17   51]]
ROC-AUC Score: 0.9480574838631104


In [35]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

ensemble_soft = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='soft'
)

ensemble_soft.fit(X_train_res, y_train_res)

y_pred = ensemble_soft.predict(X_test)
y_proba = ensemble_soft.predict_proba(X_test)[:, 1]

print("=== Soft Voting Ensemble Metrics ===")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [20:59:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


=== Soft Voting Ensemble Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.50      0.79      0.62        68

    accuracy                           0.97      2000
   macro avg       0.75      0.88      0.80      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1879   53]
 [  14   54]]
ROC-AUC: 0.9683503836317136


In [36]:
ensemble_weighted = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='soft',
    weights=[1, 2]  # XGBoost weighted higher
)

ensemble_weighted.fit(X_train_res, y_train_res)

y_pred = ensemble_weighted.predict(X_test)
y_proba = ensemble_weighted.predict_proba(X_test)[:, 1]

print("=== Weighted Ensemble Metrics ===")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [21:00:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


=== Weighted Ensemble Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.52      0.81      0.63        68

    accuracy                           0.97      2000
   macro avg       0.76      0.89      0.81      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1881   51]
 [  13   55]]
ROC-AUC: 0.9690354402630617


In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf),
        ('xgb', xgb)
    ],
    final_estimator=LogisticRegression(class_weight='balanced'),
    stack_method='predict_proba',
    n_jobs=-1
)

stacking_clf.fit(X_train_res, y_train_res)

y_pred = stacking_clf.predict(X_test)
y_proba = stacking_clf.predict_proba(X_test)[:, 1]

print("=== Stacking Ensemble Metrics ===")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


=== Stacking Ensemble Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      1932
           1       0.54      0.76      0.63        68

    accuracy                           0.97      2000
   macro avg       0.76      0.87      0.81      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1887   45]
 [  16   52]]
ROC-AUC: 0.9689136524174886


In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

dt = DecisionTreeClassifier(
    max_depth=6,                # prevents overfitting
    min_samples_leaf=20,        # stabilizes splits
    class_weight='balanced',
    random_state=42
)

dt.fit(X_train_res, y_train_res)

y_pred = dt.predict(X_test)
y_proba = dt.predict_proba(X_test)[:,1]

print("=== Decision Tree Metrics ===")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


=== Decision Tree Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1932
           1       0.31      0.85      0.45        68

    accuracy                           0.93      2000
   macro avg       0.65      0.89      0.71      2000
weighted avg       0.97      0.93      0.95      2000

Confusion Matrix:
 [[1803  129]
 [  10   58]]
ROC-AUC: 0.9459528376568019


In [39]:
ensemble_final = VotingClassifier(
    estimators=[
        ('dt', dt),
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='soft',
    weights=[1, 2, 3]
)

ensemble_final.fit(X_train_res, y_train_res)

y_pred = ensemble_final.predict(X_test)
y_proba = ensemble_final.predict_proba(X_test)[:,1]

print("=== Final Ensemble (DT + RF + XGB) Metrics ===")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [21:06:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


=== Final Ensemble (DT + RF + XGB) Metrics ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.51      0.81      0.62        68

    accuracy                           0.97      2000
   macro avg       0.75      0.89      0.80      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1879   53]
 [  13   55]]
ROC-AUC: 0.9682971014492754


In [40]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# ---------------------------------------------------
# 1. Get probabilities from FINAL model
# ---------------------------------------------------
y_proba = ensemble_weighted.predict_proba(X_test)[:, 1]

# ROC-AUC (threshold-independent, report once)
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC (Final Model):", roc_auc)

# ---------------------------------------------------
# 2. Try multiple thresholds
# ---------------------------------------------------
thresholds = [0.25, 0.30, 0.35, 0.40, 0.50]

for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)

    print("\n" + "="*50)
    print(f"Threshold = {t}")
    print("="*50)

    print(classification_report(y_test, y_pred_t))

    cm = confusion_matrix(y_test, y_pred_t)
    print("Confusion Matrix:\n", cm)

    # Optional: cost-based evaluation
    FP = cm[0, 1]
    FN = cm[1, 0]
    cost = (10 * FN) + (1 * FP)  # failure is 10x cost
    print("Estimated Maintenance Cost:", cost)


ROC-AUC (Final Model): 0.9690354402630617

Threshold = 0.25
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1932
           1       0.43      0.84      0.56        68

    accuracy                           0.96      2000
   macro avg       0.71      0.90      0.77      2000
weighted avg       0.97      0.96      0.96      2000

Confusion Matrix:
 [[1855   77]
 [  11   57]]
Estimated Maintenance Cost: 187

Threshold = 0.3
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1932
           1       0.45      0.84      0.59        68

    accuracy                           0.96      2000
   macro avg       0.72      0.90      0.78      2000
weighted avg       0.98      0.96      0.97      2000

Confusion Matrix:
 [[1863   69]
 [  11   57]]
Estimated Maintenance Cost: 179

Threshold = 0.35
              precision    recall  f1-score   support

           0       0.99      0.97      0

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# ---------------------------------------------------
# 1. Get probabilities from FINAL model
# ---------------------------------------------------
y_proba = ensemble_weighted.predict_proba(X_test)[:, 1]

# ROC-AUC (threshold-independent, report once)
roc_auc = roc_auc_score(y_test, y_proba)
print("ROC-AUC (Final Model):", roc_auc)

# ---------------------------------------------------
# 2. Try multiple thresholds
# ---------------------------------------------------
thresholds = [0.25, 0.30, 0.35, 0.40, 0.50]

for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)

    print("\n" + "="*50)
    print(f"Threshold = {t}")
    print("="*50)

    print(classification_report(y_test, y_pred_t))

    cm = confusion_matrix(y_test, y_pred_t)
    print("Confusion Matrix:\n", cm)

    # Optional: cost-based evaluation
    FP = cm[0, 1]
    FN = cm[1, 0]
    cost = (10 * FN) + (1 * FP)  # failure is 10x cost
    print("Estimated Maintenance Cost:", cost)


In [42]:
X_train_nosmote.columns = X_train_res.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
X_test_nosmote.columns = X_test.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

In [46]:
# -------- Train WITHOUT SMOTE --------
X_train_nosmote, X_test_nosmote, y_train_nosmote, y_test_nosmote = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train_nosmote.columns = X_train_nosmote.columns.astype(str)
X_test_nosmote.columns = X_test_nosmote.columns.astype(str)

# OPTIONAL but safest: remove bad characters
X_train_nosmote.columns = X_train_nosmote.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test_nosmote.columns = X_test_nosmote.columns.str.replace('[\\[\\]<]', '', regex=True)

# Random Forest with class weights
rf_cw = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42
)

# XGBoost with scale_pos_weight
scale_pos_weight = (y_train_nosmote == 0).sum() / (y_train_nosmote == 1).sum()

xgb_cw = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    random_state=42
)

# Ensemble
ensemble_cw = VotingClassifier(
    estimators=[
        ('rf', rf_cw),
        ('xgb', xgb_cw)
    ],
    voting='soft',
    weights=[2, 3]
)

ensemble_cw.fit(X_train_nosmote, y_train_nosmote)

# Probabilities
y_proba_cw = ensemble_cw.predict_proba(X_test_nosmote)[:,1]

# Apply SAME threshold
threshold = 0.35
y_pred_cw = (y_proba_cw >= threshold).astype(int)

print("=== Class-Weighted Ensemble (No SMOTE) ===")
print(classification_report(y_test_nosmote, y_pred_cw))
print("Confusion Matrix:\n", confusion_matrix(y_test_nosmote, y_pred_cw))
print("ROC-AUC:", roc_auc_score(y_test_nosmote, y_proba_cw))


=== Class-Weighted Ensemble (No SMOTE) ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1932
           1       0.74      0.79      0.77        68

    accuracy                           0.98      2000
   macro avg       0.87      0.89      0.88      2000
weighted avg       0.98      0.98      0.98      2000

Confusion Matrix:
 [[1913   19]
 [  14   54]]
ROC-AUC: 0.9736481549141396


In [47]:
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(
    n_estimators=200,
    random_state=42
)

xgb = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    random_state=42
)

ensemble_brf = VotingClassifier(
    estimators=[
        ('brf', brf),
        ('xgb', xgb)
    ],
    voting='soft',
    weights=[2, 3]
)

ensemble_brf.fit(X_train_nosmote, y_train_nosmote)

y_proba_brf = ensemble_brf.predict_proba(X_test_nosmote)[:,1]
y_pred_brf = (y_proba_brf >= 0.35).astype(int)

print("=== Balanced RF + XGB Ensemble ===")
print(classification_report(y_test_nosmote, y_pred_brf))
print("Confusion Matrix:\n", confusion_matrix(y_test_nosmote, y_pred_brf))
print("ROC-AUC:", roc_auc_score(y_test_nosmote, y_proba_brf))


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\sklearn\utils\parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fix

=== Balanced RF + XGB Ensemble ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.52      0.84      0.64        68

    accuracy                           0.97      2000
   macro avg       0.76      0.91      0.81      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1880   52]
 [  11   57]]
ROC-AUC: 0.9729859335038363


In [49]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import ADASYN

# =======================
# 2️⃣ Data Split
# =======================
# Assume your features are in X and target in y
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optional: clean column names for XGBoost
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Imbalance Handling: ADASYN
# =======================
adasyn = ADASYN(random_state=42)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)

print("After ADASYN, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models
# =======================
# Random Forest
rf_best = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

# XGBoost
scale_pos_weight = (y_train_res == 0).sum() / (y_train_res == 1).sum()
xgb_best = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Weighted Voting Ensemble
ensemble_best = VotingClassifier(
    estimators=[
        ('rf', rf_best),
        ('xgb', xgb_best)
    ],
    voting='soft',
    weights=[2, 3]
)

# =======================
# 5️⃣ Fit Ensemble on Resampled Data
# =======================
ensemble_best.fit(X_train_res, y_train_res)

# =======================
# 6️⃣ Predict Probabilities
# =======================
y_proba = ensemble_best.predict_proba(X_test)[:, 1]

# =======================
# 7️⃣ Evaluate Default Threshold (0.5)
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== Weighted Ensemble with ADASYN, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 8️⃣ Evaluate Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== Weighted Ensemble with ADASYN, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After ADASYN, class counts: [7729 7671]


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [21:45:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== Weighted Ensemble with ADASYN, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.52      0.81      0.64        68

    accuracy                           0.97      2000
   macro avg       0.76      0.89      0.81      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1882   50]
 [  13   55]]
ROC-AUC: 0.9692409572524663

=== Weighted Ensemble with ADASYN, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1932
           1       0.43      0.82      0.57        68

    accuracy                           0.96      2000
   macro avg       0.71      0.89      0.77      2000
weighted avg       0.97      0.96      0.96      2000

Confusion Matrix:
 [[1858   74]
 [  12   56]]
ROC-AUC: 0.9692409572524663


In [53]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import ADASYN

# =======================
# 2️⃣ Data Split
# =======================
# Assume your features are in X and target in y
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optional: clean column names for XGBoost
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Imbalance Handling: ADASYN
# =======================
adasyn = ADASYN(random_state=42)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)

print("After ADASYN, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models
# =======================
# Random Forest
rf_best = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

# XGBoost
scale_pos_weight = (y_train_res == 0).sum() / (y_train_res == 1).sum()
xgb_best = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

# Weighted Voting Ensemble
ensemble_best = VotingClassifier(
    estimators=[
        ('rf', rf_best),
        ('xgb', xgb_best)
    ],
    voting='soft',
    weights=[1, 4]
)

# =======================
# 5️⃣ Fit Ensemble on Resampled Data
# =======================
ensemble_best.fit(X_train_res, y_train_res)

# =======================
# 6️⃣ Predict Probabilities
# =======================
y_proba = ensemble_best.predict_proba(X_test)[:, 1]

# =======================
# 7️⃣ Evaluate Default Threshold (0.5)
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== Weighted Ensemble with ADASYN, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 8️⃣ Evaluate Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== Weighted Ensemble with ADASYN, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After ADASYN, class counts: [7729 7671]


c:\Users\Swetha Pooduru\Desktop\Predictive Maintenance Machine Failure Risk Classification\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [21:50:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== Weighted Ensemble with ADASYN, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.53      0.81      0.64        68

    accuracy                           0.97      2000
   macro avg       0.76      0.89      0.81      2000
weighted avg       0.98      0.97      0.97      2000

Confusion Matrix:
 [[1883   49]
 [  13   55]]
ROC-AUC: 0.9701924247960054

=== Weighted Ensemble with ADASYN, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1932
           1       0.46      0.82      0.59        68

    accuracy                           0.96      2000
   macro avg       0.73      0.89      0.78      2000
weighted avg       0.98      0.96      0.97      2000

Confusion Matrix:
 [[1866   66]
 [  12   56]]
ROC-AUC: 0.9701924247960054


In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# ----------------------
# 1️⃣ Resample with ADASYN
# ----------------------
adasyn = ADASYN(random_state=42)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)

# ----------------------
# 2️⃣ Define Models
# ----------------------
dt = DecisionTreeClassifier(
    max_depth=5,  # optional tuning
    random_state=42
)

scale_pos_weight = (y_train_res == 0).sum() / (y_train_res == 1).sum()
xgb = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    random_state=42
)

# ----------------------
# 3️⃣ Weighted Voting Ensemble
# ----------------------
# Give more weight to DT (e.g., DT=3, XGB=2)
ensemble_dt_xgb = VotingClassifier(
    estimators=[('dt', dt), ('xgb', xgb)],
    voting='soft',
    weights=[3, 2]
)

ensemble_dt_xgb.fit(X_train_res, y_train_res)

# ----------------------
# 4️⃣ Predict Probabilities
# ----------------------
y_proba = ensemble_dt_xgb.predict_proba(X_test)[:,1]

# ----------------------
# 5️⃣ Default Threshold 0.5
# ----------------------
y_pred_default = (y_proba >= 0.5).astype(int)
print("=== DT+XGB Ensemble, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# ----------------------
# 6️⃣ Risk-Optimized Threshold 0.35
# ----------------------
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+XGB Ensemble, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


=== DT+XGB Ensemble, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.94      0.97      1932
           1       0.34      0.85      0.48        68

    accuracy                           0.94      2000
   macro avg       0.66      0.90      0.72      2000
weighted avg       0.97      0.94      0.95      2000

Confusion Matrix:
 [[1817  115]
 [  10   58]]
ROC-AUC: 0.9359624284496407

=== DT+XGB Ensemble, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.91      0.95      1932
           1       0.26      0.88      0.41        68

    accuracy                           0.91      2000
   macro avg       0.63      0.90      0.68      2000
weighted avg       0.97      0.91      0.93      2000

Confusion Matrix:
 [[1764  168]
 [   8   60]]
ROC-AUC: 0.9359624284496407


In [55]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# =======================
# 2️⃣ Data Split
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optional: clean column names for XGBoost
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Imbalance Handling: SMOTE
# =======================
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models
# =======================
# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)

# XGBoost
scale_pos_weight = (y_train_res == 0).sum() / (y_train_res == 1).sum()
xgb = XGBClassifier(
    n_estimators=200,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    random_state=42
)

# =======================
# 5️⃣ Weighted Voting Ensemble
# =======================
# Give more weight to DT (DT=3, XGB=2)
ensemble_dt_xgb = VotingClassifier(
    estimators=[('dt', dt), ('xgb', xgb)],
    voting='soft',
    weights=[3, 2]
)

# =======================
# 6️⃣ Fit Ensemble
# =======================
ensemble_dt_xgb.fit(X_train_res, y_train_res)

# =======================
# 7️⃣ Predict Probabilities
# =======================
y_proba = ensemble_dt_xgb.predict_proba(X_test)[:, 1]

# =======================
# 8️⃣ Evaluate Default Threshold 0.5
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== DT+XGB Ensemble with SMOTE, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Risk-Optimized Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+XGB Ensemble with SMOTE, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After SMOTE, class counts: [7729 7729]

=== DT+XGB Ensemble with SMOTE, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      1932
           1       0.37      0.85      0.52        68

    accuracy                           0.95      2000
   macro avg       0.68      0.90      0.74      2000
weighted avg       0.97      0.95      0.96      2000

Confusion Matrix:
 [[1833   99]
 [  10   58]]
ROC-AUC: 0.9550222262818171

=== DT+XGB Ensemble with SMOTE, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1932
           1       0.29      0.88      0.43        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.69      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1782  150]
 [   8   60]]
ROC-AUC: 0.9550222262818171


In [56]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# =======================
# 2️⃣ Data Split
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optional: clean column names for XGBoost / RF
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Imbalance Handling: SMOTE
# =======================
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models
# =======================
# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)

# =======================
# 5️⃣ Weighted Voting Ensemble
# =======================
# Give more weight to DT (DT=3, RF=2)
ensemble_dt_rf = VotingClassifier(
    estimators=[('dt', dt), ('rf', rf)],
    voting='soft',
    weights=[3, 2]
)

# =======================
# 6️⃣ Fit Ensemble
# =======================
ensemble_dt_rf.fit(X_train_res, y_train_res)

# =======================
# 7️⃣ Predict Probabilities
# =======================
y_proba = ensemble_dt_rf.predict_proba(X_test)[:, 1]

# =======================
# 8️⃣ Evaluate Default Threshold 0.5
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== DT+RF Ensemble with SMOTE, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Risk-Optimized Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+RF Ensemble with SMOTE, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After SMOTE, class counts: [7729 7729]

=== DT+RF Ensemble with SMOTE, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1932
           1       0.30      0.85      0.44        68

    accuracy                           0.93      2000
   macro avg       0.65      0.89      0.70      2000
weighted avg       0.97      0.93      0.94      2000

Confusion Matrix:
 [[1797  135]
 [  10   58]]
ROC-AUC: 0.9563352514919011

=== DT+RF Ensemble with SMOTE, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1932
           1       0.28      0.88      0.43        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.69      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1778  154]
 [   8   60]]
ROC-AUC: 0.9563352514919011


In [57]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# =======================
# 2️⃣ Data Split
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optional: clean column names for XGBoost / RF
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Imbalance Handling: SMOTE
# =======================
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models
# =======================
# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)

# =======================
# 5️⃣ Weighted Voting Ensemble
# =======================
# Give more weight to DT (DT=3, RF=2)
ensemble_dt_rf = VotingClassifier(
    estimators=[('dt', dt), ('rf', rf)],
    voting='soft',
    weights=[4, 2]
)

# =======================
# 6️⃣ Fit Ensemble
# =======================
ensemble_dt_rf.fit(X_train_res, y_train_res)

# =======================
# 7️⃣ Predict Probabilities
# =======================
y_proba = ensemble_dt_rf.predict_proba(X_test)[:, 1]

# =======================
# 8️⃣ Evaluate Default Threshold 0.5
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== DT+RF Ensemble with SMOTE, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Risk-Optimized Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+RF Ensemble with SMOTE, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After SMOTE, class counts: [7729 7729]

=== DT+RF Ensemble with SMOTE, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1932
           1       0.29      0.87      0.44        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.70      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1789  143]
 [   9   59]]
ROC-AUC: 0.9552772195834857

=== DT+RF Ensemble with SMOTE, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1932
           1       0.28      0.88      0.43        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.69      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1780  152]
 [   8   60]]
ROC-AUC: 0.9552772195834857


In [58]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# =======================
# 2️⃣ Data Split
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optional: clean column names for XGBoost / RF
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
X_train.columns = X_train.columns.str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Imbalance Handling: SMOTE
# =======================
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models
# =======================
# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)

# =======================
# 5️⃣ Weighted Voting Ensemble
# =======================
# Give more weight to DT (DT=3, RF=2)
ensemble_dt_rf = VotingClassifier(
    estimators=[('dt', dt), ('rf', rf)],
    voting='soft',
    weights=[5, 2]
)

# =======================
# 6️⃣ Fit Ensemble
# =======================
ensemble_dt_rf.fit(X_train_res, y_train_res)

# =======================
# 7️⃣ Predict Probabilities
# =======================
y_proba = ensemble_dt_rf.predict_proba(X_test)[:, 1]

# =======================
# 8️⃣ Evaluate Default Threshold 0.5
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== DT+RF Ensemble with SMOTE, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Risk-Optimized Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+RF Ensemble with SMOTE, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After SMOTE, class counts: [7729 7729]

=== DT+RF Ensemble with SMOTE, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1932
           1       0.29      0.87      0.44        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.70      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1789  143]
 [   9   59]]
ROC-AUC: 0.9537624832541712

=== DT+RF Ensemble with SMOTE, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1932
           1       0.28      0.88      0.42        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.69      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1777  155]
 [   8   60]]
ROC-AUC: 0.9537624832541712


In [59]:
from itertools import product
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE

# ----------------------
# 1️⃣ Resample with SMOTE
# ----------------------
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# ----------------------
# 2️⃣ Define base models
# ----------------------
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
rf = RandomForestClassifier(n_estimators=200, random_state=42)

# ----------------------
# 3️⃣ Grid search for weights & thresholds
# ----------------------
dt_weights = [3, 4, 5]
rf_weights = [2, 1]
thresholds = [0.25, 0.3, 0.35, 0.4, 0.5]

results = []

for dw, rw in product(dt_weights, rf_weights):
    ensemble = VotingClassifier(
        estimators=[('dt', dt), ('rf', rf)],
        voting='soft',
        weights=[dw, rw]
    )
    ensemble.fit(X_train_res, y_train_res)
    y_proba = ensemble.predict_proba(X_test)[:, 1]
    
    for th in thresholds:
        y_pred = (y_proba >= th).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        recall = cm[1,1] / (cm[1,0]+cm[1,1])
        f1 = classification_report(y_test, y_pred, output_dict=True)['1']['f1-score']
        fp = cm[0,1]
        fn = cm[1,0]
        
        results.append({
            'DT_weight': dw,
            'RF_weight': rw,
            'Threshold': th,
            'Recall': recall,
            'F1': f1,
            'FP': fp,
            'FN': fn
        })

# ----------------------
# 4️⃣ Convert to DataFrame
# ----------------------
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='Recall', ascending=False)
print(df_results)


    DT_weight  RF_weight  Threshold    Recall        F1   FP  FN
0           3          2       0.25  0.911765  0.418919  166   6
5           3          1       0.25  0.911765  0.420339  165   6
25          5          1       0.25  0.911765  0.421769  164   6
20          5          2       0.25  0.911765  0.423208  163   6
10          4          2       0.25  0.911765  0.418919  166   6
15          4          1       0.25  0.911765  0.421769  164   6
6           3          1       0.30  0.897059  0.425087  158   7
11          4          2       0.30  0.897059  0.423611  159   7
21          5          2       0.30  0.897059  0.426573  157   7
16          4          1       0.30  0.897059  0.423611  159   7
26          5          1       0.30  0.897059  0.423611  159   7
1           3          2       0.30  0.897059  0.419244  162   7
27          5          1       0.35  0.897059  0.431095  154   7
12          4          2       0.35  0.882353  0.428571  152   8
13          4          2 

In [60]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# =======================
# 2️⃣ Data Split
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Clean column names
X_train.columns = X_train.columns.astype(str).str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.astype(str).str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Handle Imbalance with SMOTE
# =======================
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Hyperparameter tuning for DT
# =======================
dt_params = {
    'max_depth': [5, 6, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'criterion': ['gini', 'entropy']
}

dt_search = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_distributions=dt_params,
    n_iter=10,
    scoring='recall',
    cv=3,
    random_state=42
)
dt_search.fit(X_train_res, y_train_res)
best_dt = dt_search.best_estimator_
print("Best DT params:", dt_search.best_params_)

# =======================
# 5️⃣ Hyperparameter tuning for RF
# =======================
rf_params = {
    'n_estimators': [200, 300, 400],
    'max_depth': [None, 7, 10],
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced']
}

rf_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=rf_params,
    n_iter=10,
    scoring='recall',
    cv=3,
    random_state=42
)
rf_search.fit(X_train_res, y_train_res)
best_rf = rf_search.best_estimator_
print("Best RF params:", rf_search.best_params_)

# =======================
# 6️⃣ Weighted Voting Ensemble
# =======================
ensemble = VotingClassifier(
    estimators=[('dt', best_dt), ('rf', best_rf)],
    voting='soft',
    weights=[5, 2]  # DT dominates
)
ensemble.fit(X_train_res, y_train_res)

# =======================
# 7️⃣ Predict Probabilities
# =======================
y_proba = ensemble.predict_proba(X_test)[:,1]

# =======================
# 8️⃣ Evaluate Threshold 0.5
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== DT+RF Ensemble (Tuned) Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+RF Ensemble (Tuned) Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After SMOTE, class counts: [7729 7729]
Best DT params: {'min_samples_split': 6, 'min_samples_leaf': 2, 'max_depth': 5, 'criterion': 'entropy'}
Best RF params: {'n_estimators': 400, 'max_features': 'log2', 'max_depth': None, 'class_weight': 'balanced'}

=== DT+RF Ensemble (Tuned) Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.92      0.96      1932
           1       0.29      0.87      0.43        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90      0.70      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1787  145]
 [   9   59]]
ROC-AUC: 0.9578994641334795

=== DT+RF Ensemble (Tuned) Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1932
           1       0.28      0.88      0.42        68

    accuracy                           0.92      2000
   macro avg       0.64      0.90   

In [61]:
# =======================
# 1️⃣ Imports
# =======================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# =======================
# 2️⃣ Data Split
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Clean column names
X_train.columns = X_train.columns.astype(str).str.replace('[\\[\\]<]', '', regex=True)
X_test.columns = X_test.columns.astype(str).str.replace('[\\[\\]<]', '', regex=True)

# =======================
# 3️⃣ Handle Imbalance with ADASYN
# =======================
adasyn = ADASYN(random_state=42)
X_train_res, y_train_res = adasyn.fit_resample(X_train, y_train)
print("After ADASYN, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Define Models with Class Weights
# =======================
# Heavier weight for minority class
dt = DecisionTreeClassifier(max_depth=5, class_weight={0:1, 1:10}, random_state=42)
rf = RandomForestClassifier(n_estimators=200, class_weight={0:1, 1:10}, random_state=42)

# =======================
# 5️⃣ Weighted Voting Ensemble
# =======================
ensemble = VotingClassifier(
    estimators=[('dt', dt), ('rf', rf)],
    voting='soft',
    weights=[5, 2]  # DT dominates
)
ensemble.fit(X_train_res, y_train_res)

# =======================
# 6️⃣ Predict Probabilities
# =======================
y_proba = ensemble.predict_proba(X_test)[:,1]

# =======================
# 7️⃣ Evaluate Threshold 0.5
# =======================
y_pred_default = (y_proba >= 0.5).astype(int)
print("\n=== DT+RF Ensemble with ADASYN, Threshold 0.5 ===")
print(classification_report(y_test, y_pred_default))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_default))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 8️⃣ Evaluate Threshold 0.35
# =======================
threshold = 0.35
y_pred_thresh = (y_proba >= threshold).astype(int)
print("\n=== DT+RF Ensemble with ADASYN, Threshold 0.35 ===")
print(classification_report(y_test, y_pred_thresh))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Threshold 0.3 (more aggressive)
# =======================
threshold = 0.3
y_pred_thresh2 = (y_proba >= threshold).astype(int)
print("\n=== DT+RF Ensemble with ADASYN, Threshold 0.3 ===")
print(classification_report(y_test, y_pred_thresh2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh2))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After ADASYN, class counts: [7729 7671]

=== DT+RF Ensemble with ADASYN, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      1932
           1       0.17      0.93      0.29        68

    accuracy                           0.85      2000
   macro avg       0.58      0.88      0.60      2000
weighted avg       0.97      0.85      0.89      2000

Confusion Matrix:
 [[1629  303]
 [   5   63]]
ROC-AUC: 0.9436502862014372

=== DT+RF Ensemble with ADASYN, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       1.00      0.76      0.86      1932
           1       0.12      0.93      0.21        68

    accuracy                           0.76      2000
   macro avg       0.56      0.84      0.54      2000
weighted avg       0.97      0.76      0.84      2000

Confusion Matrix:
 [[1463  469]
 [   5   63]]
ROC-AUC: 0.9436502862014372

=== DT+RF Ensemble with ADASYN, Threshold 0.3 ===
    

In [62]:
# =======================
# 1️⃣ Imports
# =======================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# =======================
# 2️⃣ Split Data
# =======================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# =======================
# 3️⃣ Handle Imbalance with SMOTE
# =======================
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE, class counts:", np.bincount(y_train_res))

# =======================
# 4️⃣ Scale Features for SVM
# =======================
scaler = StandardScaler()
X_train_res_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

# =======================
# 5️⃣ Define Models
# =======================
svm_model = SVC(probability=True, class_weight='balanced', random_state=42)
dt_model = DecisionTreeClassifier(class_weight={0:1, 1:5}, random_state=42)

# =======================
# 6️⃣ Weighted Voting Ensemble
# =======================
ensemble = VotingClassifier(
    estimators=[('svm', svm_model), ('dt', dt_model)],
    voting='soft',
    weights=[5, 2]  # more weight to SVM
)
ensemble.fit(X_train_res_scaled, y_train_res)

# =======================
# 7️⃣ Predict Probabilities
# =======================
y_proba = ensemble.predict_proba(X_test_scaled)[:,1]

# =======================
# 8️⃣ Evaluate Threshold 0.5
# =======================
threshold = 0.5
y_pred = (y_proba >= threshold).astype(int)
print("\n=== SVM+DT Ensemble, Threshold 0.5 ===")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# =======================
# 9️⃣ Evaluate Threshold 0.35
# =======================
threshold = 0.35
y_pred = (y_proba >= threshold).astype(int)
print("\n=== SVM+DT Ensemble, Threshold 0.35 ===")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))


After SMOTE, class counts: [7729 7729]

=== SVM+DT Ensemble, Threshold 0.5 ===
              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1932
           1       0.28      0.75      0.40        68

    accuracy                           0.92      2000
   macro avg       0.63      0.84      0.68      2000
weighted avg       0.97      0.92      0.94      2000

Confusion Matrix:
 [[1798  134]
 [  17   51]]
ROC-AUC: 0.9519318597004018

=== SVM+DT Ensemble, Threshold 0.35 ===
              precision    recall  f1-score   support

           0       0.99      0.91      0.95      1932
           1       0.25      0.87      0.39        68

    accuracy                           0.91      2000
   macro avg       0.62      0.89      0.67      2000
weighted avg       0.97      0.91      0.93      2000

Confusion Matrix:
 [[1757  175]
 [   9   59]]
ROC-AUC: 0.9519318597004018
